# Settings, Directory Specs, and Imports

In [1]:
do_plots = 1 # 1: create plots

dir_read = '/Users/rachellehorwitz/Documents/ViTalErt/data/filtered05/' # directory to read data from

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, confusion_matrix, plot_confusion_matrix, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, learning_curve, train_test_split, ShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import check_random_state
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from datetime import datetime
import pyprojroot
from pandas_profiling import ProfileReport
from datetime import datetime
from sklearn.model_selection._split import _BaseKFold
from sklearn.model_selection._split import _RepeatedSplits
from collections import defaultdict
from collections import Counter
import sys
sys.path.insert(1, '../src')
from utils import *

pd.set_option('display.max_columns', None)

matplotlib.rcParams.update({'font.size': 33})

%matplotlib qt

# Data Wrangling

In [2]:
# Read files from csv
patient = pd.read_csv(dir_read + 'patient.csv')
diagnosis = pd.read_csv(dir_read + 'diagnosis.csv')
ptusid_pos = pd.read_csv(dir_read + 'pos_ptunitstayid.csv')
hr = pd.read_csv(dir_read + 'vitalPeriodic_heartrate_first24.csv')
hr = hr.set_index('patientunitstayid')

print('There are ' + str(patient.shape[0]) + ' patientunitstayids in "patient.csv" but only ' \
      + str(hr.shape[0]) + ' patientunitstayids in the hr csv file')
print('Cleaning that up for patient, diagnosis, and ptusid_pos.')
patient = patient[patient['patientunitstayid'].isin(list(hr.index))]
diagnosis = diagnosis[diagnosis['patientunitstayid'].isin(list(hr.index))]
ptusid_pos = ptusid_pos[ptusid_pos['patientunitstayid'].isin(list(hr.index))]

hr = hr.reset_index()

There are 161543 patientunitstayids in "patient.csv" but only 158784 patientunitstayids in the hr csv file
Cleaning that up for patient, diagnosis, and ptusid_pos.


In [3]:
# Create dataframe of patientunitstayids in negative class
tmp = patient[~patient['patientunitstayid'].isin(ptusid_pos['patientunitstayid'])]
ptusid_neg = tmp[['patientunitstayid']]

# Create labels
ptusid_neg = ptusid_neg.assign(label=0)
ptusid_pos = ptusid_pos.assign(label=1)

# Add labels to "patient" dataframe and move 'label' column to left
ptusid_w_label = pd.concat([ptusid_pos, ptusid_neg])
ptusid_w_label.shape[0]
ptusid_w_label = ptusid_w_label.set_index('patientunitstayid')
patient = patient.set_index('patientunitstayid')
patient = patient.join(ptusid_w_label)

# Move 'label' column to left
cols = list(patient.columns)
cols = [cols[-1]] + cols[:-1]
patient = patient[cols]

# Add BMI column to patient
patient = patient.assign(bmi=lambda x: x.admissionweight/((x.admissionheight/100)**2))

# Remove patients who weigh less than 40 kg or more than 300 kg
patient = patient[(patient['admissionweight'] > 40) & \
                   (patient['admissionweight'] < 300)]

# Remove patients who are shorter than 100 cm (3.2 ft) or taller than 230 cm (~7.5 ft)
patient = patient[(patient['admissionheight'] > 100 ) & (patient['admissionheight'] < 230)]

# Remove patients with BMI > 100
patient = patient[patient['bmi'] < 100]

# Dummy variables for gender, ethnicity, unitstaytype
patient = pd.concat([patient, pd.get_dummies(patient['gender'], prefix='gender')], axis=1)
patient = pd.concat([patient, pd.get_dummies(patient['ethnicity'], prefix='ethnicity')], axis=1)
patient = pd.concat([patient, pd.get_dummies(patient['unitstaytype'], prefix='unitstaytype')], axis=1)

nstds = 5
print(str(169+nstds*patient['admissionheight'].std()))
print(str(169-nstds*patient['admissionheight'].std()))

patient_pos = patient[patient['label']==1]
patient_neg = patient[patient['label']==0]
patient[patient['label']==1].shape[0]

# Drop unneeded columns of patient
patient = patient.drop(columns=['hospitalid', 'wardid', 'dischargeweight', 'unitdischargetime24', \
                                'unitdischargeoffset', 'unitdischargelocation', 'hospitaldischargeyear',\
                               'hospitaldischargetime24', 'hospitaldischargelocation'])

223.22514749375495
114.77485250624505


In [4]:
patient.shape

(156992, 33)

In [5]:
# ApachePredVar
u_ptunitstayid = list(patient.index)
apachepredvar = pd.read_csv('/Users/rachellehorwitz/Documents/ViTalErt/data/over18_eicu/apachePredVar.csv')
apachepredvar = apachepredvar[apachepredvar['patientunitstayid'].isin(u_ptunitstayid)]

# Drop columns based on apachepredvar_allcols.html
apachepredvar = apachepredvar.drop(columns=['apachepredvarid', 'sicuday', 'saps3day1', 'saps3yesterday', 'saps3today', 'gender', \
                                    'teachtype', 'region', 'amilocation', 'bedcount', 'admitsource', 'graftcount', 'age', \
                                   'meds', 'diedinhospital', 'electivesurgery', 'readmit', 'managementsystem', 'var03hspxlos', \
                                   'ejectfx', 'day1meds', 'day1verbal', 'day1motor', 'day1eyes',  'day1pao2', 'day1fio2', \
                                           'ventday1', 'creatinine', 'dischargelocation', 'pao2', 'fio2']) # pao2 and fio2 are <0 too frequently
apachepredvar = apachepredvar.drop(columns=['admitdiagnosis'])
apachepredvar = apachepredvar[apachepredvar['verbal']!=-1] # Get rid of missing data
apachepredvar.set_index('patientunitstayid')

,verbal,motor,eyes,thrombolytics,aids,hepaticfailure,lymphoma,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes,visitnumber
patientunitstayid,,,,,,,,,,,,,,,,,,
141168,5,6,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
141194,4,6,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
141197,5,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
141203,1,3,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1
141208,5,6,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353201,5,6,3,0,0,0,0,0,0,0,0,1,0,0,1,1,0,3
3353216,1,5,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1
3353235,5,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# Define X and y

In [6]:
Xy = patient[['uniquepid', 'label',  'age', 'admissionweight', 'admissionheight', 'bmi', 'gender_Female',
       'ethnicity_African American', 'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_Native American', 'ethnicity_Other/Unknown']]
Xy = Xy.merge(apachepredvar, how='right', left_on='patientunitstayid', right_on='patientunitstayid')
Xy = Xy.merge(hr, how='inner', left_on='patientunitstayid', right_on='patientunitstayid')

# Arrange by categorical & continuous variables
vars_categ = ['gender_Female', 'ethnicity_African American', 'ethnicity_Asian', 'ethnicity_Caucasian', \
             'ethnicity_Hispanic', 'ethnicity_Native American', 'ethnicity_Other/Unknown',\
             'thrombolytics', 'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia', \
             'immunosuppression', 'cirrhosis', 'activetx', 'ima', 'midur',
             'oobventday1', 'oobintubday1', 'diabetes']
vars_cont = ['age', 'admissionweight', 'admissionheight', 'bmi', \
             'verbal', 'motor', 'eyes', 'visitnumber', 'heartrate']
Xy = pd.concat([Xy[['patientunitstayid', 'uniquepid', 'label']], Xy[vars_cont], Xy[vars_categ]], axis=1)

In [7]:
Xy.head()

,patientunitstayid,uniquepid,label,age,admissionweight,admissionheight,bmi,verbal,motor,eyes,visitnumber,heartrate,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,thrombolytics,aids,hepaticfailure,lymphoma,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes
0,141168,002-34851,0,70,84.3,152.4,36.295906,5,6,4,1,125.052830,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,141194,002-5276,0,68,73.9,180.3,22.732803,4,6,3,1,86.860627,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,141197,002-37665,0,71,102.1,162.6,38.617545,5,6,4,1,97.307692,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,141203,002-23234,0,77,70.2,160.0,27.421875,1,3,1,1,91.543554,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
4,141208,002-34521,0,25,95.3,172.7,31.952749,5,6,3,1,77.817460,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Describe postive and negative classes
print('Columns:')
print(Xy.columns)
print('--------------------------')
print('Total: ' + str(Xy.shape[0]) + ' unique patientunitstayids from ' + \
     str(Xy.groupby('uniquepid')['patientunitstayid'].count().shape[0]) + ' unique patients')
print('Negative class: ' + str(Xy[Xy['label']==0].shape[0]) + ' unique patientunitstayids from ' + \
     str(Xy[Xy['label']==0].groupby('uniquepid')['patientunitstayid'].count().shape[0]) + ' unique patients')
print('Positive class: ' + str(Xy[Xy['label']==1].shape[0]) + ' unique patientunitstayids from ' + \
     str(Xy[Xy['label']==1].groupby('uniquepid')['patientunitstayid'].count().shape[0]) + ' unique patients')


# How many patients are in both classes?
Xy1 = Xy[Xy['label']==1]
Xy0 = Xy[Xy['label']==0]
upid_1 = Xy1['uniquepid'].unique()
upid_1 = list(upid_1)
patients_in_both = Xy0[Xy0['uniquepid'].isin(upid_1)]
print('There are ' + str(len(patients_in_both)) + ' patients in both classes')
print('--------------------------')
# Describe the features
print('There are ' + str(len(vars_categ)) + ' categorical features')
print('There are ' + str(len(vars_cont)) + ' continuous features')

# Separate X and y
Xy = Xy.set_index('patientunitstayid')
y = Xy.pop('label')
X = Xy.copy()
groups = Xy['uniquepid'].astype('category').cat.codes # each uniquepid is now a unique number
X = X.drop(columns='uniquepid', axis=1) # remove uniquepid as a feature because it's a group
X.head()

Columns:
Index(['patientunitstayid', 'uniquepid', 'label', 'age', 'admissionweight',
       'admissionheight', 'bmi', 'verbal', 'motor', 'eyes', 'visitnumber',
       'heartrate', 'gender_Female', 'ethnicity_African American',
       'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_Native American', 'ethnicity_Other/Unknown', 'thrombolytics',
       'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia',
       'immunosuppression', 'cirrhosis', 'activetx', 'ima', 'midur',
       'oobventday1', 'oobintubday1', 'diabetes'],
      dtype='object')
--------------------------
Total: 146242 unique patientunitstayids from 118016 unique patients
Negative class: 145790 unique patientunitstayids from 117723 unique patients
Positive class: 452 unique patientunitstayids from 447 unique patients
There are 268 patients in both classes
--------------------------
There are 21 categorical features
There are 9 continuous features


,age,admissionweight,admissionheight,bmi,verbal,motor,eyes,visitnumber,heartrate,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,thrombolytics,aids,hepaticfailure,lymphoma,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes
patientunitstayid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141168,70,84.3,152.4,36.295906,5,6,4,1,125.052830,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
141194,68,73.9,180.3,22.732803,4,6,3,1,86.860627,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
141197,71,102.1,162.6,38.617545,5,6,4,1,97.307692,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
141203,77,70.2,160.0,27.421875,1,3,1,1,91.543554,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
141208,25,95.3,172.7,31.952749,5,6,3,1,77.817460,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Split into 80/20 train&val/test using StratifiedGroupKFold()
cv = StratifiedGroupKFold(n_splits = 5, shuffle=True, random_state=1)
trainval_idx, test_idx = next(cv.split(X, y, groups))
X_trainval = X.iloc[trainval_idx]
y_trainval = y.iloc[trainval_idx]
X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]

In [10]:
Xy_trainval = pd.concat([y_trainval, X_trainval], axis=1)
Xy_trainval_pos = Xy_trainval[Xy_trainval['label']==1]
Xy_trainval_neg = Xy_trainval[Xy_trainval['label']==0]
X_trainval_pos = Xy_trainval_pos.drop(columns='label')
X_trainval_neg = Xy_trainval_neg.drop(columns='label')

In [11]:
#profile = ProfileReport(Xy_trainval, title='Pandas Profiling Report')
#profile.to_file("Xy_trainval_profilereport.html")

In [12]:
corr_map = Xy_trainval.corr()
corr_map

,label,age,admissionweight,admissionheight,bmi,verbal,motor,eyes,visitnumber,heartrate,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,thrombolytics,aids,hepaticfailure,lymphoma,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes
label,1.000000,-0.000617,0.013054,0.004248,0.011625,-0.026844,-0.023445,-0.024858,0.010728,0.027497,-0.004501,0.006442,-0.002347,-0.002431,0.000432,-0.001385,-0.003152,-0.002471,-0.001946,-0.000035,0.002766,0.004728,-0.001413,0.007320,0.000064,0.025715,-0.006355,-0.003865,0.032385,0.033330,-0.007206
age,-0.000617,1.000000,-0.147906,-0.130625,-0.101310,-0.016282,0.026249,0.025893,0.016156,-0.170701,0.037612,-0.124292,-0.000806,0.144555,-0.024119,-0.055407,-0.053267,0.006991,-0.031874,-0.034441,0.018120,0.019829,0.028009,0.008858,-0.042880,0.075856,0.034155,0.023114,0.025096,-0.023356,0.029711
admissionweight,0.013054,-0.147906,1.000000,0.376644,0.891538,0.018121,0.007302,-0.003319,0.006688,-0.015915,-0.232643,0.040116,-0.084291,0.031383,-0.043031,0.005261,-0.033472,0.016706,-0.017646,-0.003965,-0.010340,-0.037600,-0.011326,-0.028950,-0.000869,0.061244,0.039275,0.003001,0.084800,0.051480,0.146153
admissionheight,0.004248,-0.130625,0.376644,1.000000,-0.061975,0.006746,-0.009651,-0.007662,0.004492,-0.026053,-0.674200,0.034968,-0.088663,0.067103,-0.077110,-0.013228,-0.054743,0.024014,0.004741,0.010911,0.005059,0.005076,0.005825,0.000555,0.012893,-0.003667,0.040697,0.007867,0.001369,0.023344,-0.012111
bmi,0.011625,-0.101310,0.891538,-0.061975,1.000000,0.015530,0.012032,-0.000984,0.006177,-0.005771,0.056728,0.031988,-0.055927,0.002309,-0.012966,0.011239,-0.011207,0.006776,-0.019178,-0.008693,-0.013407,-0.041712,-0.014642,-0.030515,-0.006512,0.066977,0.022796,-0.000195,0.090952,0.044938,0.164421
verbal,-0.026844,-0.016282,0.018121,0.006746,0.015530,1.000000,0.689175,0.760745,-0.051324,-0.092005,0.002057,-0.015736,0.005738,0.012663,0.010780,-0.014621,-0.008437,0.038633,0.001093,-0.008152,0.005511,0.016996,0.006381,0.023800,-0.012199,-0.329836,-0.062113,0.008113,-0.516583,-0.601802,0.025594
motor,-0.023445,0.026249,0.007302,-0.009651,0.012032,0.689175,1.000000,0.782810,-0.024169,-0.062809,0.009618,-0.025285,-0.003235,0.022433,0.011528,-0.011268,-0.010065,0.009713,-0.000684,-0.008872,0.008827,0.016486,0.005306,0.022402,-0.007561,-0.247785,-0.045555,0.000338,-0.397625,-0.466101,0.023273
eyes,-0.024858,0.025893,-0.003319,-0.007662,-0.000984,0.760745,0.782810,1.000000,-0.028478,-0.068411,0.002098,-0.019393,0.005050,0.009928,0.025638,-0.021274,-0.007815,0.025411,-0.000257,-0.012189,0.009156,0.015030,0.001124,0.021740,-0.014022,-0.278306,-0.042332,0.003646,-0.441294,-0.507915,0.024953
visitnumber,0.010728,0.016156,0.006688,0.004492,0.006177,-0.051324,-0.024169,-0.028478,1.000000,0.047501,-0.008437,0.003964,0.003051,-0.000625,-0.019574,0.004175,0.009213,-0.016905,0.000989,0.021031,0.019535,0.004322,0.005413,0.022409,0.023521,0.077742,0.015884,0.022490,0.068278,0.046101,0.008595
heartrate,0.027497,-0.170701,-0.015915,-0.026053,-0.005771,-0.092005,-0.062809,-0.068411,0.047501,1.000000,0.049656,0.045811,0.000042,-0.041047,0.005951,0.013158,0.001678,-0.075839,0.018872,0.023981,0.022034,0.048221,0.021941,0.065659,0.020947,0.128198,-0.000685,-0.017434,0.110585,0.089131,0.009178


In [13]:
corrmap_bigcorrs = corr_map[corr_map >0.9]
corr_map.where(corr_map>0.8)

,label,age,admissionweight,admissionheight,bmi,verbal,motor,eyes,visitnumber,heartrate,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,thrombolytics,aids,hepaticfailure,lymphoma,metastaticcancer,leukemia,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes
label,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admissionweight,NaN,NaN,1.000000,NaN,0.891538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admissionheight,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bmi,NaN,NaN,0.891538,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
verbal,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
motor,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitnumber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heartrate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Plots

In [14]:
import seaborn as sns

X_trainval_pos = X_trainval[y_trainval==1]
X_trainval_neg = X_trainval[y_trainval==0]

fig = plt.figure()
VTE = sns.distplot(X_trainval_pos['admissionweight'], color='red', bins=20)
noVTE = sns.distplot(X_trainval_neg['admissionweight'], color='blue', bins=20)
plt.xlabel('Admission Weight (kg)')
fig.legend(labels=['VTE', 'No VTE'])

fig = plt.figure()
sns.distplot(X_trainval_pos['age'], color='red', bins=20)
sns.distplot(X_trainval_neg['age'], color='blue', bins=20)
fig.legend(labels=['VTE', 'No VTE'])
plt.xlabel('Age (yrs)')

fig = plt.figure()
sns.distplot(X_trainval_pos['bmi'], color='red', bins=20)
sns.distplot(X_trainval_neg['bmi'], color='blue', bins=20)
plt.xlabel('BMI (kg/m^2)')

fig = plt.figure()
sns.distplot(X_trainval_pos['admissionheight'], color='red', bins=20)
sns.distplot(X_trainval_neg['admissionheight'], color='blue', bins=20)
plt.xlabel('Height (cm))')

plt.rcParams.update({'font.size': 40})
sns.set(font_scale = 1.25)
fig = plt.figure()
sns.distplot(X_trainval_pos['heartrate'], color='red', bins=20, hist=False, label='VTE')
sns.distplot(X_trainval_neg['heartrate'], color='blue', bins=20, hist=False, label='No VTE')
plt.xlabel('Average Heart Rate \nDuring First 24 Hours (beats/min)')
plt.ylabel('Density')
plt.tight_layout()

In [15]:
# Table of each variable's average value 
Xy_trainval_categ = pd.concat([Xy_trainval['label'], Xy_trainval[vars_categ]], axis=1)
table = pd.pivot_table(data=Xy_trainval_categ, index=['label'])
tablet = table.transpose()
tablet = tablet.rename(columns = {0: 'No VTE', 1: 'VTE'})
tablet = tablet.rename(index={'activetx': 'Active \n Treatment', 'metastaticcancer': 'Metastatic \n Cancer', 'oobventday1': 'Ventilated'})
tablet['VTE'] = tablet['VTE']
tablet['No VTE'] = tablet['No VTE']
tablet

label,No VTE,VTE
Active \n Treatment,0.583090,0.811634
aids,0.001226,0.000000
cirrhosis,0.019231,0.019391
diabetes,0.241160,0.185596
ethnicity_African American,0.110399,0.146814
ethnicity_Asian,0.016462,0.011080
ethnicity_Caucasian,0.779932,0.761773
ethnicity_Hispanic,0.037305,0.038781
ethnicity_Native American,0.007725,0.005540
ethnicity_Other/Unknown,0.048177,0.036011


In [16]:
# Scatter plot of some of the features
# Randomly sample 200 data points from each class
nsamp = 200
Xy_pos_nsamp = Xy_trainval[Xy_trainval['label']==1].sample(n = nsamp)
Xy_neg_nsamp = Xy_trainval[Xy_trainval['label']==0].sample(n = nsamp)
Xy_nsamp = pd.concat([Xy_pos_nsamp, Xy_neg_nsamp])
vars_compare = ['age', 'admissionweight', 'admissionheight', 'bmi', 'heartrate']


In [17]:
cols = list(Xy_nsamp.columns)
cols_new = ['label', 'age', 'admissionweight', 'admissionheight', 'bmi', 'heartrate']
Xy_nsamp_cont = Xy_nsamp[cols_new]


In [18]:
ax = sns.pairplot(Xy_nsamp_cont, hue='label')